<a href="https://colab.research.google.com/github/maxenlee/DataScienceToolBox/blob/main/BigQueryTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install google-cloud-bigquery


In [2]:
from google.colab import auth
auth.authenticate_user()


In [3]:
# !wget -O ToolBox.py 'https://raw.githubusercontent.com/maxenlee/DataScienceToolBox/split/ToolBox.py'
!wget -O ToolBox.py 'https://raw.githubusercontent.com/maxenlee/DataScienceToolBox/main/ToolBox.py'


--2024-03-05 01:36:49--  https://raw.githubusercontent.com/maxenlee/DataScienceToolBox/main/ToolBox.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7373 (7.2K) [text/plain]
Saving to: ‘ToolBox.py’

ToolBox.py          100%[===================>]   7.20K  --.-KB/s    in 0s      

2024-03-05 01:36:49 (61.4 MB/s) - ‘ToolBox.py’ saved [7373/7373]



In [4]:

from google.cloud import bigquery
import pandas as pd
import datetime
import logging
from google.colab import userdata


# Set your Google Cloud Project ID from secrets on the side bar
project_id = userdata.get('project_id')
billing_project_id = project_id

# Create client object
client = bigquery.Client(project=billing_project_id)

# Specify the dataset ID here
dataset_id = 'stackoverflow'

sub_project = "bigquery-public-data"

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset(dataset_id, project=sub_project)

# For use in queries
source = f'{sub_project}.{dataset_id}'

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)


In [5]:
from ToolBox import configure_bigquery, bigquery
configure_bigquery(project_id=project_id)


# Gemini

## 1. Install Required Libraries:

In [6]:
source

'bigquery-public-data.stackoverflow'

In [7]:
# !pip install google-cloud-bigquery


###2 Import the Toolbox:

## Test 1: Standard Query Execution
This test executes a simple query against the Stack Overflow dataset and displays the results. No DataFrame variable is specified, so the results should be displayed directly.

In [8]:
%%bigquery
SELECT id, title, creation_date
FROM `{source}.posts_questions`
WHERE tags LIKE '%python%'
LIMIT 5


2024-03-05 01:36:53,493 - BigQueryMagic - ERROR - GoogleAPIError: 404 Not found: Dataset cryptic-idiom-414323:default-source was not found in location US

Location: US
Job ID: dbf899e8-cfa3-49c9-b0c6-b3c74d336223



ERROR:BigQueryMagic:GoogleAPIError: 404 Not found: Dataset cryptic-idiom-414323:default-source was not found in location US

Location: US
Job ID: dbf899e8-cfa3-49c9-b0c6-b3c74d336223



## Test 2: Query Execution with Results Stored in a DataFrame
This test executes a query and stores the results in a DataFrame named df_questions.

In [9]:
%%bigquery df_questions
SELECT id, title, creation_date
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags LIKE '%python%'
LIMIT 5

Query results stored in DataFrame 'df_questions'.


In [10]:
df_questions

,id,title,creation_date
0,3540479,How do I modify a datastream on the fly?,2010-08-22 06:41:32.237000+00:00
1,3222018,Google wave authentication login on 3rd party ...,2010-07-11 05:47:29.130000+00:00
2,3238150,How can I delete an attribute when a flush or ...,2010-07-13 14:25:08.783000+00:00
3,3241078,Error in IPython Shell,2010-07-13 20:11:23.603000+00:00
4,3269830,How to copy an App Engine Datastore entity,2010-07-17 01:27:24.680000+00:00


## Test 3: Dry Run to Estimate Query Cost
This test performs a dry run of a query to estimate its cost without executing it. No results are fetched, and no DataFrame is created.



In [11]:
%%bigquery dry
SELECT id, title, creation_date
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags LIKE '%python%'
LIMIT 5


Dry run: Estimated bytes to be processed: 2305621912 bytes.
Estimated cost of the query: $0.01


## Test 4: Overriding source and project_id in the Magic Command
This test demonstrates overriding the source and project_id directly in the magic command line. Replace <your-project-id> with your actual Google Cloud project ID. Note that this functionality depends on how you implemented the handling of these optional arguments in the bigquery function.

In [12]:
%%bigquery --source=bigquery-public-data.stackoverflow --project_id=cryptic-idiom-414323
SELECT id, title, creation_date
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags LIKE '%java%'
LIMIT 5


,id,title,creation_date
0,3232809,bitshift equivalent of math.round,2010-07-12 22:34:26.247000+00:00
1,3238525,How to add timer to a blackberry application,2010-07-13 15:07:00.773000+00:00
2,3246513,Displaying Timer for blackberry app,2010-07-14 13:23:03.450000+00:00
3,3429078,String Manipulation required,2010-08-07 03:58:01.967000+00:00
4,3435709,Why are functions in JavaScript set to global ...,2010-08-08 19:19:48.390000+00:00


## Test 5: Error Handling with an Invalid Query
This test intentionally uses an invalid query to demonstrate how the cell magic handles and displays errors.

In [13]:
%%bigquery
SELECT nonexistent_column
FROM `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 5


2024-03-05 01:36:59,941 - BigQueryMagic - ERROR - GoogleAPIError: 400 Unrecognized name: nonexistent_column at [1:8]

Location: US
Job ID: cc5a9288-b2b1-40f1-9f12-a787bac1397e



ERROR:BigQueryMagic:GoogleAPIError: 400 Unrecognized name: nonexistent_column at [1:8]

Location: US
Job ID: cc5a9288-b2b1-40f1-9f12-a787bac1397e



### Test 6:Exporting Results to a File
To write the query results directly to a file:

In [14]:
%%bigquery --output_file="results.csv"
SELECT id, title, creation_date
FROM `{source}.posts_questions`
WHERE tags LIKE '%python%'
LIMIT 5


Query results stored in /content/results.csv
